# 青空文庫の本文から文豪の言及を抽出する

In [1]:
from bs4 import BeautifulSoup
import re
import codecs

# 対象のファイルの全文データとメタデータをハッシュで返す
def get_text(path):
    try:
      with codecs.open(path, 'r', 'shift-jis', 'ignore') as f:
          text = f.read()
          soup = BeautifulSoup(text, 'html.parser')
        
          title = soup.find('h1').text
          author_name = soup.find('h2').text
          main_text_block = soup.find('div', class_='main_text')
        
          # ルビを除去して本文を取得
          [ rb.decompose() for rb in main_text_block.findAll('rp') ]
          [ rb.decompose() for rb in main_text_block.findAll('rt') ]
          main_text = main_text_block.text.replace('\n', '').replace('\r', '').replace('\u3000', ' ')
        
          main_text
    except:
      print("【ERROR】{0}".format(path))
      raise 
    
    return {
        'path': path,
        'title': title,
        'author_name': author_name,
        'text': main_text
    }

get_text("./aozora/cards/000035/files/2257_15061.html")

{'path': './aozora/cards/000035/files/2257_15061.html',
 'title': '黄金風景',
 'author_name': '太宰治',
 'text': '海の岸辺に緑なす樫の木、その樫の木に黄金の細き鎖のむすばれて   ―プウシキン― 私は子供のときには、余り質のいい方ではなかった。女中をいじめた。私は、のろくさいことは嫌いで、それゆえ、のろくさい女中を殊にもいじめた。お慶は、のろくさい女中である。林檎の皮をむかせても、むきながら何を考えているのか、二度も三度も手を休めて、おい、とその度毎にきびしく声を掛けてやらないと、片手に林檎、片手にナイフを持ったまま、いつまでも、ぼんやりしているのだ。足りないのではないか、と思われた。台所で、何もせずに、ただのっそりつっ立っている姿を、私はよく見かけたものであるが、子供心にも、うすみっともなく、妙に疳にさわって、おい、お慶、日は短いのだぞ、などと大人びた、いま思っても脊筋の寒くなるような非道の言葉を投げつけて、それで足りずに一度はお慶をよびつけ、私の絵本の観兵式の何百人となくうようよしている兵隊、馬に乗っている者もあり、旗持っている者もあり、銃担っている者もあり、そのひとりひとりの兵隊の形を鋏でもって切り抜かせ、不器用なお慶は、朝から昼飯も食わず日暮頃までかかって、やっと三十人くらい、それも大将の鬚を片方切り落したり、銃持つ兵隊の手を、熊の手みたいに恐ろしく大きく切り抜いたり、そうしていちいち私に怒鳴られ、夏のころであった、お慶は汗かきなので、切り抜かれた兵隊たちはみんな、お慶の手の汗で、びしょびしょ濡れて、私は遂に癇癪をおこし、お慶を蹴った。たしかに肩を蹴った筈なのに、お慶は右の頬をおさえ、がばと泣き伏し、泣き泣きいった。「親にさえ顔を踏まれたことはない。一生おぼえております」うめくような口調で、とぎれ、とぎれそういったので、私は、流石にいやな気がした。そのほかにも、私はほとんどそれが天命でもあるかのように、お慶をいびった。いまでも、多少はそうであるが、私には無智な魯鈍の者は、とても堪忍できぬのだ。 一昨年、私は家を追われ、一夜のうちに窮迫し、巷をさまよい、諸所に泣きつき、その日その日のいのち繋ぎ、やや文筆でもって、自活できるあてがつきはじめたと思ったとたん、病を得

## 文豪リストを読み込む

メンバーが作成した文豪のリストを読み込む

In [2]:
import pandas as pd
import json

def read_bungo():
  with open('bungo.json', 'r') as f:
    return json.load(f)

bungo = read_bungo()
bungo

[{'ndla': '00000800',
  'viewName': '会津 八一',
  'portraitID': '',
  'normName': '会津, 八一, 1881-1956',
  'aozora': 'https://www.aozora.gr.jp/index_pages/person1245.html',
  'ndlThumb': '',
  'ndlaUrl': 'http://id.ndl.go.jp/auth/entity/00000800',
  'jpsUrl': 'https://jpsearch.go.jp/entity/chname/会津八一',
  'comment': '会津 八一（あいづ やいち、會津八一、1881年（明治14年）8月1日 - 1956年（昭和31年）11月21日）は、日本の歌人・美術史家・書家。雅号は、秋艸道人、渾斎。1951年に新潟市名誉市民。'},
 {'ndla': '00001305',
  'viewName': '秋田 雨雀',
  'portraitID': '',
  'normName': '秋田, 雨雀, 1883-1962',
  'aozora': 'https://www.aozora.gr.jp/index_pages/person1584.html',
  'ndlThumb': '',
  'ndlaUrl': 'http://id.ndl.go.jp/auth/entity/00001305',
  'jpsUrl': '',
  'comment': '秋田 雨雀（あきた うじゃく、1883年（明治16年）1月30日 - 1962年（昭和37年）5月12日）は、日本の劇作家・詩人・童話作家・小説家である。本名は徳三（とくぞう）。'},
 {'ndla': '00001509',
  'viewName': '芥川 竜之介',
  'portraitID': '224',
  'normName': '芥川, 竜之介, 1892-1927',
  'aozora': 'https://www.aozora.gr.jp/index_pages/person879.html',
  'ndlThumb': 'https://www.ndl.go.jp/portrait

## 青空文庫の作品リストを取得する

In [3]:
import pandas

def read_worklist():
  return pandas.read_csv('list_person_all_extended_utf8.csv', dtype = {"人物ID": "object"})

# サンプル: 会津八一の作品リストを抽出する
works = read_worklist()
works[works["人物ID"] == "001245"]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,作品ID,作品名,作品名読み,ソート用読み,副題,副題読み,原題,初出,分類番号,文字遣い種別,...,テキストファイルURL,テキストファイル最終更新日,テキストファイル符号化方式,テキストファイル文字集合,テキストファイル修正回数,XHTML/HTMLファイルURL,XHTML/HTMLファイル最終更新日,XHTML/HTMLファイル符号化方式,XHTML/HTMLファイル文字集合,XHTML/HTMLファイル修正回数
5,46511,一片の石,いっぺんのいし,いつへんのいし,NaN,NaN,NaN,NaN,NDC 914,新字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0
6,57975,音楽に就いて,おんがくについて,おんかくについて,NaN,NaN,NaN,「興風」1922（大正11）年2月,NDC 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/57...,2017-10-25,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/57...,2017-10-25,ShiftJIS,JIS X 0208,0.0
7,57976,学規,がくき,かくき,NaN,NaN,NaN,NaN,NDC 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/57...,2017-06-13,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/57...,2017-06-13,ShiftJIS,JIS X 0208,0.0
8,46512,菊の根分をしながら,きくのねわけをしながら,きくのねわけをしなから,NaN,NaN,NaN,NaN,NDC 914,新字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0
9,46513,支那の明器,しなのめいき,しなのめいき,NaN,NaN,NaN,NaN,NDC 914,新字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/46...,2007-01-01,ShiftJIS,JIS X 0208,0.0
10,57977,趣味の向上,しゅみのこうじょう,しゆみのこうしよう,――青年学生のために――,――せいねんがくせいのために――,NaN,「興風」1924（大正13）年12月,NDC 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/57...,2017-10-25,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/57...,2017-10-25,ShiftJIS,JIS X 0208,0.0
11,58663,趣味の修養,しゅみのしゅうよう,しゆみのしゆうよう,NaN,NaN,NaN,「興風」1922（大正11）年8月,NDC 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/58...,2018-10-24,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/58...,2018-10-24,ShiftJIS,JIS X 0208,0.0
12,57978,少年少女におくる言葉,しょうねんしょうじょにおくることば,しようねんしようしよにおくることは,NaN,NaN,NaN,「少年少女新潟日報」1952（昭和27）年1月13日,NDC K914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/57...,2017-06-13,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/57...,2017-06-13,ShiftJIS,JIS X 0208,0.0
13,58664,綜合大学の図書,そうごうだいがくのとしょ,そうこうたいかくのとしよ,NaN,NaN,NaN,「夕刊ニイガタ」1948（昭和23）年5月25日,NDC 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/58...,2019-07-30,ShiftJIS,JIS X 0208,0.0,https://www.aozora.gr.jp/cards/001245/files/58...,2019-07-30,ShiftJIS,JIS X 0208,0.0
14,46514,拓本の話,たくほんのはなし,たくほんのはなし,NaN,NaN,NaN,NaN,NDC 728 914,旧字旧仮名,...,https://www.aozora.gr.jp/cards/001245/files/46...,2012-04-11,ShiftJIS,JIS X 0208,1.0,https://www.aozora.gr.jp/cards/001245/files/46...,2012-04-11,ShiftJIS,JIS X 0208,1.0


In [4]:
import glob
import re

def get_author_id(author_url):
  pattern = r'person([0-9]+)'
  m = re.search(pattern, str(author_url))
  if m:
    author_id = "{0:06d}".format(int(m.group(1)))
  else:
    author_id = None

  return author_id

def get_text_paths_by_author(author_id, works = None):
  if works == None:
    works = read_worklist()

  author_worklist = works[ works["人物ID"] == author_id ]
  return [ url.replace('https://www.aozora.gr.jp', './aozora') for url in author_worklist["XHTML/HTMLファイルURL"]  if re.search(r'aozora', str(url)) ]

def get_author_textpaths(author_url):
  author_id = get_author_id(author_url)
  if author_id:
    textpaths = get_text_paths_by_author(author_id)
  else:
    textpaths = None
  
  return textpaths

for row in read_bungo()[0:2]:
  author_url = row["aozora"]
  author_texts = [ get_text(textpath) for textpath in get_author_textpaths(author_url) ]
  data = {
    "author": dict(row),
    "author_url": author_url,
    "texts": author_texts
  }
  print(data["author"])
  print(len(data["texts"]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.


{'ndla': '00000800', 'viewName': '会津 八一', 'portraitID': '', 'normName': '会津, 八一, 1881-1956', 'aozora': 'https://www.aozora.gr.jp/index_pages/person1245.html', 'ndlThumb': '', 'ndlaUrl': 'http://id.ndl.go.jp/auth/entity/00000800', 'jpsUrl': 'https://jpsearch.go.jp/entity/chname/会津八一', 'comment': '会津 八一（あいづ やいち、會津八一、1881年（明治14年）8月1日 - 1956年（昭和31年）11月21日）は、日本の歌人・美術史家・書家。雅号は、秋艸道人、渾斎。1951年に新潟市名誉市民。'}
10
{'ndla': '00001305', 'viewName': '秋田 雨雀', 'portraitID': '', 'normName': '秋田, 雨雀, 1883-1962', 'aozora': 'https://www.aozora.gr.jp/index_pages/person1584.html', 'ndlThumb': '', 'ndlaUrl': 'http://id.ndl.go.jp/auth/entity/00001305', 'jpsUrl': '', 'comment': '秋田 雨雀（あきた うじゃく、1883年（明治16年）1月30日 - 1962年（昭和37年）5月12日）は、日本の劇作家・詩人・童話作家・小説家である。本名は徳三（とくぞう）。'}
1


## MongoDBを導入

In [5]:
!pip install pymongo

In [6]:
import pymongo

def db():
  client = pymongo.MongoClient('mongodb://mongodb:27017/')
  return client.aozoratext

## 青空文庫の本文データをMongoDBに登録

In [11]:
aozora_db = db()


def save_db(row):
  error_texts = list()

  author_url = row["aozora"]
  print(author_url)
  author_textpaths = get_author_textpaths(author_url)

  if author_textpaths:
    print("{0}の登録開始".format(row["viewName"]))

    texts = list()
    
    for textpath in author_textpaths:
      try:
        text = get_text(textpath)
        text['viewName'] = row['viewName']
        texts.append(get_text(textpath))
      except:
        error_texts.append(textpath)
        print("【ERROR】{0}".format(textpath))
    
    aozora_db.text.insert_many(texts)
    
    print("{0}の登録完了: {1} 件".format(row["viewName"], len(texts)))
    
  else:
    print("no texts")

  return error_texts

error_texts = list()
for row in bungo:
  error_texts = error_texts + save_db(bungo[87])

with open('error_texts.txt', 'a') as f:
  for path in error_texts:
    f.write("{0}\n".format(path))

https://www.aozora.gr.jp/index_pages/person1562.html


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  


吉川 英治の登録開始
吉川 英治の登録完了: 130 件


In [84]:
db().text.count()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


10127

In [24]:
from itertools import groupby

def get_refer_to(author_name):
  return list(db().text.find({'text': {'$regex': author_name} }))

def group_get_refer_to(author_name):
  referdata = get_refer_to(author_name)
  result    = dict()
  
  for work in referdata:
    author_id = re.search(r'cards/([0-9]+)', work['path']).group(1)
    
    if author_id not in result:
      result[author_id] = { 'size': 0, 'works': list() }
    result[author_id]['size'] += 1
    result[author_id]['works'].append(work)

  return result

def get_refer_all():
  referdata = dict()
  for author in read_bungo():
    print(author['viewName'])
    data = group_get_refer_to(author['viewName'].replace(' ', ''))
    print(data.keys())
    referdata[author['viewName']] = data
  return referdata

referdata = get_refer_all()

会津 八一
dict_keys(['001095', '001562'])
秋田 雨雀
dict_keys(['001154', '000255', '000311'])
芥川 竜之介
dict_keys(['001569', '000067', '001154', '000042', '000906'])
有島 武郎
dict_keys(['000879', '000035', '001569', '001154', '000311', '000040', '001095', '000158', '001383', '000214', '000885', '001562'])
太宰 治
dict_keys(['000035', '001154', '000311', '000040', '001095', '001763', '000906'])
江戸川 乱歩
dict_keys(['001670', '001779', '000255', '000216', '001095', '000160', '000096'])
中谷 宇吉郎
dict_keys(['000311', '001670', '001095'])
萩原 朔太郎
dict_keys(['000879', '001779', '000067', '001030', '000255', '000311', '001579', '001095'])
原 民喜
dict_keys(['000293'])
服部 誠一
dict_keys(['001341', '000388'])
樋口 一葉
dict_keys(['000050', '000083', '000183', '000311', '001341', '000283', '001670', '000082', '001095', '001168'])
堀 辰雄
dict_keys(['000879', '000293', '001030', '001154', '000183', '000311', '001579', '000040', '000933', '001763'])
福地 源一郎
dict_keys(['000083', '000311', '000129', '001670', '000076', '000082', '0013

dict_keys(['000879', '000035', '000074', '000083', '001154', '000311', '000119', '000906', '001562'])
横光 利一
dict_keys(['000879', '000035', '001569', '001030', '000074', '001154', '000183', '000311', '000119', '000216', '000040', '000076', '001095', '001310', '000214', '000906', '000168', '001562'])
与謝野 晶子
dict_keys(['000879', '000025', '001154', '000311', '000129', '001341', '000216', '001670', '000040', '001763', '000885', '001562'])
吉井 勇
dict_keys(['000879', '000035', '000153', '000050', '001154', '000106', '000183', '001341', '000933', '001168', '000162', '000320', '001562'])
吉川 英治
dict_keys(['000311', '000216', '001670', '001095', '001562'])
夢野 久作
dict_keys(['000255', '000096'])
三木 露風
dict_keys(['000067', '000106', '000076', '000933'])
坂本 竜馬
dict_keys(['000255', '000283', '000148'])
淡島 寒月
dict_keys(['001341'])
コナン・ドイル
dict_keys(['000035', '001670', '001779', '001569', '000067', '001154', '000216', '000076', '000082'])
ルイス・キャロル
dict_keys(['001393'])
小泉 八雲
dict_keys(['001245', '00087

In [47]:
import json

with open('referdata.json', 'w') as f:
  dumpdata = dict()

  for name in referdata:
    dumpdata[name] = dict()
    for refername in referdata[name]:
      works = [ {'title': work['title'], 'path': work['path']} for work in referdata[name][refername]['works'] ]
      dumpdata[name][refername] = {'size': referdata[name][refername]['size'], 'works': works}

  json.dump(dumpdata, f,  ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))

In [83]:
def get_author_id_from_url(url):
  m = re.search(r'person([0-9]+)', url)
  if m:
    return "{0:06d}".format(int(m.group(1)))
  else:
    return ''

def format_referdata():
  rule = [{'viewName': author['viewName'], 'ndla': author['ndla'], 'author_id': get_author_id_from_url(author['aozora']) } for author in read_bungo() ]
  viewNamedic = { item['viewName']:item['ndla'] for item in rule }
  author_iddic = { item['author_id']:item['ndla'] for item in rule }

  def size(data):
    if 'size' in data:
      return data['size']
    else:
      return 0

  with open('referdata.json', 'r') as f:
    referdata = json.load(f)
    formatdata = list()
    for viewName in referdata:
      try:
        item = {'id': viewNamedic[viewName], 'relation': [ {'id': author_iddic[i], 'score':size(referdata[viewName][i]) } for i in referdata[viewName].keys() ] }
        formatdata.append(item)
      except:
        path
        
  return formatdata

finaldata = format_referdata()
with open('score.json', 'w') as f:
  json.dump(finaldata, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))